# Train an SVM classifier on the MNIST dataset. 

Since SVM classifiers are binary classifiers, you will need to use one-versus-all to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?_

First, let's load the dataset and split it into a training set and a test set. We could use `train_test_split()` but people usually just take the first 60,000 instances for the training set, and the last 10,000 instances for the test set (this makes it possible to compare your model's performance with others): 
Don't forget to convert target to int.

In [10]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform


In [2]:
train_data = pd.read_csv('mnist_train.csv')
test_data = pd.read_csv('mnist_test.csv')

In [3]:
test_data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train = train_data.drop('label', axis=1)
y_train = train_data.label.astype('int')
X_test = test_data.drop('label', axis=1)
y_test = test_data.label.astype('int')

In [5]:
y_test

0       7
1       2
2       1
3       0
4       4
       ..
9995    2
9996    3
9997    4
9998    5
9999    6
Name: label, Length: 10000, dtype: int32

Many training algorithms are sensitive to the order of the training instances, so it's generally good practice to shuffle them first. However, the dataset is already shuffled, so we do not need to do it.

Let's start simple, with a linear SVM classifier. It will automatically use the One-vs-All (also called One-vs-the-Rest, OvR) strategy, so there's nothing special we need to do. Use a LinearSVC and initialize it with a randon_state to make results reproducible.

**Warning**: this may take a few minutes depending on your hardware.

In [6]:
lin_svc = svm.LinearSVC(random_state=42)
lin_svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

Let's make predictions on the training set and measure the accuracy (we don't want to measure it on the test set yet, since we have not selected and trained the final model yet):

In [7]:
y_pred = lin_svc.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_pred, y_test))


Accuracy: 0.8832


Okay, Probably you got an accuracy bellow 90%. On MNIST that is pretty bad. This linear model is certainly too simple for MNIST, but perhaps we just needed to scale the data first. Use and StandardScaler to do it:
Hint: take into account that you should convert the features to float. 

In [8]:
X_train = X_train.astype(float)
X_train.shape

(60000, 784)

In [11]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
X_train_scl = scaler.transform(X_train)
X_test_scl = scaler.transform(X_test)

In [13]:
X_train_scl

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Try to train the model again on the scaled features.
**Warning**: this may take a few minutes depending on your hardware.

Let's make predictions on the training set and measure the accuracy 

In [14]:
lin_svc.fit(X_train_scl, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [15]:
y_pred_scl = lin_svc.predict(X_test_scl)

In [16]:
print("Accuracy:",metrics.accuracy_score(y_pred_scl, y_test))

Accuracy: 0.9132


0.9225333333333333

That's much better (we cut the error rate by about 25%), but still not great at all for MNIST. If we want to use an SVM, we will have to use a kernel. Let's try an `SVC` with an RBF kernel (the default).
#### **Let's train the model only on 10000 samples**  

In [17]:
X_train_tenthou = X_train.iloc[:10000]
y_train_tenthou = y_train.iloc[:10000]
X_train_tenthou.shape

(10000, 784)

**Note**: to be future-proof set `gamma="scale"` since it will be the default value in Scikit-Learn 0.22.


In [18]:
# Create a SVC classifier using a linear kernel
svm = SVC(gamma='scale')
# Train the classifier
svm.fit(X_train_tenthou, y_train_tenthou)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Let's make predictions on the training set and measure the accuracy 

In [20]:
y_pred_RBF = svm.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_pred_RBF, y_test))

Accuracy: 0.9594


0.9455333333333333

That's promising, we get better performance even though we trained the model on 6 times less data. Let's tune the hyperparameters by doing a randomized search with cross validation. We will do this on a small dataset (1000 samples) just to speed up the process:
Use a RandomizedSearchCV

In [ ]:
X_train_thou = X_train.iloc[:1000]
y_train_thou = y_train.iloc[:1000]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
estimator_RBF = SVC(gamma='scale')
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}

CVrandom_RBF = RandomizedSearchCV(estimator=estimator_RBF, param_distributions=param_distributions, random_state=42)
search_RBF = CVrandom_RBF.fit(X_train_thou, y_train_thou)
search_RBF.best_params_


Let's print the best estimator and the best score

In [ ]:
best = search_RBF.best_estimator_

In [ ]:
search_RBF.best_score_

This looks pretty low but remember we only trained the model on 1,000 instances. Let's retrain the best estimator on the whole training set (**run this at night, it will take hours**):

In [ ]:
# Train the classifier
best.fit(X_train, y_train)

Let's make predictions on the training set and measure the accuracy 

In [ ]:
y_pred_best = best.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_pred_best, y_test))

Ah, this looks good! Let's select this model. Now we can test it on the test set:

In [ ]:
y_pred_test = best.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_pred_test, y_test))

Not too bad, but apparently the model is overfitting slightly. It's tempting to tweak the hyperparameters a bit more (e.g. decreasing `C` and/or `gamma`), but we would run the risk of overfitting the test set. Other people have found that the hyperparameters `C=5` and `gamma=0.005` yield even better performance (over 98% accuracy). By running the randomized search for longer and on a larger part of the training set, you may be able to find this as well.

## Train an SVM regressor on the California housing dataset

Let's load the dataset using Scikit-Learn's `fetch_california_housing()` function:

In [54]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC

In [39]:
from sklearn.datasets import fetch_california_housing 
cali = fetch_california_housing(data_home=None, download_if_missing=True, return_X_y=False)
print(cali.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [40]:
cali

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n--------------------

In [41]:
X = pd.DataFrame(cali.data, columns=cali['feature_names'])
y = cali.target
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [42]:
#Sin df
'''X = cali.data
y = cali.target'''

'X = cali.data\ny = cali.target'

In [43]:
X.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude'],
      dtype='object')

In [44]:
y

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

Split it into a training set and a test (20%) set:

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
type(y_train)

numpy.ndarray

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  float64
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB


In [47]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 14196 to 15795
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      16512 non-null  float64
 1   HouseAge    16512 non-null  float64
 2   AveRooms    16512 non-null  float64
 3   AveBedrms   16512 non-null  float64
 4   Population  16512 non-null  float64
 5   AveOccup    16512 non-null  float64
 6   Latitude    16512 non-null  float64
 7   Longitude   16512 non-null  float64
dtypes: float64(8)
memory usage: 1.1 MB


Don't forget to scale the data. Use a StandardScaler

In [48]:
scaler = StandardScaler()

In [49]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
X_train_scaled

array([[-0.326196  ,  0.34849025, -0.17491646, ...,  0.05137609,
        -1.3728112 ,  1.27258656],
       [-0.03584338,  1.61811813, -0.40283542, ..., -0.11736222,
        -0.87669601,  0.70916212],
       [ 0.14470145, -1.95271028,  0.08821601, ..., -0.03227969,
        -0.46014647, -0.44760309],
       ...,
       [-0.49697313,  0.58654547, -0.60675918, ...,  0.02030568,
        -0.75500738,  0.59946887],
       [ 0.96545045, -1.07984112,  0.40217517, ...,  0.00707608,
         0.90651045, -1.18553953],
       [-0.68544764,  1.85617335, -0.85144571, ..., -0.08535429,
         0.99543676, -1.41489815]])

In [51]:
type(X_train_scaled)

numpy.ndarray

In [52]:
y_train

array([1.03 , 3.821, 1.726, ..., 2.221, 2.835, 3.25 ])

Let's train a simple `LinearSVR` first:

In [56]:
#lin_svc = SVC(kernel ='linear')
lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)
y_pred_lin = lin_svr.predict(X_test_scaled)
lin_svr

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=42, tol=0.0001, verbose=0)

Let's see how it performs on the training set using the Mean Squared Error:

In [57]:
mean_squared_error(y_test, y_pred_lin)

0.5816018911362225

Let's look at the RMSE:

In [58]:
mean_squared_error(y_test, y_pred_lin, squared=False)

0.7626282784792487

In [59]:
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred_lin))

print(rmse)

0.7626282784792487


In [60]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred_lin)

0.5561676539205584

In this training set, the targets are tens of thousands of dollars. The RMSE gives a rough idea of the kind of error you should expect (with a higher weight for large errors): so with this model we can expect errors somewhere around $10,000. Not great. Let's see if we can do better with an RBF Kernel. We will use randomized search with cross validation to find the appropriate hyperparameter values for `C` and `gamma`:

In [62]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
from scipy.stats import randint
estimator_RBF = SVR()
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}

CVrandom_RBF = RandomizedSearchCV(estimator=estimator_RBF, param_distributions=param_distributions, random_state=42)
search_RBF = CVrandom_RBF.fit(X_train_scaled, y_train)
search_RBF.best_params_

{'C': 4.745401188473625, 'gamma': 0.07969454818643928}

In [63]:
'''from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train_scaled, y_train)'''

"from sklearn.ensemble import RandomForestRegressor\nfrom sklearn.model_selection import RandomizedSearchCV\n\nparam_distribs = {\n        'n_estimators': randint(low=1, high=200),\n        'max_features': randint(low=1, high=8),\n    }\n\nforest_reg = RandomForestRegressor(random_state=42)\nrnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,\n                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)\nrnd_search.fit(X_train_scaled, y_train)"

Let's see the best estimator.

In [64]:
search_RBF.best_estimator_

SVR(C=4.745401188473625, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.07969454818643928, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

Now let's measure the RMSE  and the squared error on the training set for this estimator:

In [65]:
best = search_RBF.best_estimator_
best.fit(X_train_scaled, y_train)
y_pred_best = best.predict(X_test_scaled)

In [66]:
mean_squared_error(y_test, y_pred_best)

0.351561654701232

In [67]:
mean_squared_error(y_test, y_pred_best, squared=False)

0.5929263484626333

In [69]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred_best)

0.7317160821248567

Looks much better than the linear model. Let's select this model and evaluate it on the test set:

In [139]:
'''best.fit(X_test_scaled, y_test)
y_pred_test = best.predict(X_test_scaled)
mean_squared_error(y_test, y_pred_test)'''

0.3112217535523372

In [142]:
'''from sklearn.metrics import r2_score
r2_score(y_test, y_pred_test)'''

0.7625002890547018